## Usage of one explainer

Take MTER as an example, and use dataset amazon_toy.

In [1]:
from cornac.datasets import amazon_toy
from cornac.data.reader import Reader
from cornac.eval_methods import StratifiedSplit
from cornac.data.sentiment import SentimentModality
from cornac.models import MTER
from cornac.explainer import Exp_MTER

# Load the Amazon toy dataset
rating = amazon_toy.load_feedback(fmt="UIRT", reader=Reader(min_user_freq=50))
sentiment_data = amazon_toy.load_sentiment(reader=Reader(min_user_freq=50))

md = SentimentModality(data=sentiment_data)

eval_method = StratifiedSplit(
    data=rating,
    group_by="user",
    chrono=True,
    sentiment=md,
    test_size=0.2,
    exclude_unknowns=True,
    verbose=True,
)

# Instantiate the MTER model
mter = MTER(
    max_iter=20,
    n_user_factors=8,
    n_item_factors=8,
    n_aspect_factors=8,
    n_opinion_factors=8,
    n_bpr_samples=1000,
    n_element_samples=50,
    lambda_reg=0.1,
    lambda_bpr=10,
    lr=0.5,
)

mter.fit(eval_method.train_set)

# Instantiate the explainer
explainer = Exp_MTER(mter, eval_method.train_set)


/home/linan/miniconda3/envs/cornac/lib/python3.11/site-packages/tensorflow/python/debug/cli/debugger_cli_common.py:19: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants


rating_threshold = 1.0
exclude_unknowns = True
---
Training data:
Number of users = 119
Number of items = 4058
Number of ratings = 7197
Max rating = 5.0
Min rating = 1.0
Global mean = 4.3
---
Test data:
Number of users = 119
Number of items = 4058
Number of ratings = 741
Number of unknown users = 0
Number of unknown items = 0
---
Total users = 119
Total items = 4058


### Explain recommendations

In [8]:
# first, we need some recommendations
some_users = eval_method.test_set.user_ids[:3]
recomedations = mter.recommend_to_multiple_users(some_users, k=5)
print(recomedations)

           user_id     item_id  prediction
0   A30H2335OM7RD6  B006ROJJE6    1.137697
1   A30H2335OM7RD6  B00CKH9URU    1.089992
2   A30H2335OM7RD6  B000284ZNI    1.068191
3   A30H2335OM7RD6  B00BN6QAVI    1.052156
4   A30H2335OM7RD6  B00CI6J6UY    1.048574
5   A3R01WHD75L6FG  B006ROJJE6    0.521874
6   A3R01WHD75L6FG  B00CI6J6UY    0.507830
7   A3R01WHD75L6FG  B00CKH9URU    0.504350
8   A3R01WHD75L6FG  B002B555QQ    0.492050
9   A3R01WHD75L6FG  B000284ZNI    0.484729
10  A19FRW264WZTGP  B006ROJJE6    3.214391
11  A19FRW264WZTGP  B00CKH9URU    3.085872
12  A19FRW264WZTGP  B00CI6J6UY    3.036532
13  A19FRW264WZTGP  B000284ZNI    2.991767
14  A19FRW264WZTGP  B00BN6QAVI    2.969527


In [10]:
explainer.explain_recommendations(recomedations)

Computing explanations:   0%|          | 0/15 [00:00<?, ?it/s]

,user_id,item_id,prediction,explanations
0,A30H2335OM7RD6,B006ROJJE6,1.137697,"{'pieces': {'damage': 1.1116805, 'excellent': ..."
1,A30H2335OM7RD6,B00CKH9URU,1.089992,"{'pieces': {'neat': 1.5739713, 'attack': 0.657..."
2,A30H2335OM7RD6,B000284ZNI,1.068191,"{'back': {'loves': 0.1975783, 'instantly': 0.1..."
3,A30H2335OM7RD6,B00BN6QAVI,1.052156,"{'also': {'adore': 0.7846245, 'lose': 0.543023..."
4,A30H2335OM7RD6,B00CI6J6UY,1.048574,"{'fact': {'problems': 0.2618758, 'creative': 0..."
5,A3R01WHD75L6FG,B006ROJJE6,0.521874,"{'pieces': {'damage': 0.38749737, 'excellent':..."
6,A3R01WHD75L6FG,B00CI6J6UY,0.507830,"{'fact': {'problems': 0.09688483, 'creative': ..."
7,A3R01WHD75L6FG,B00CKH9URU,0.504350,"{'pieces': {'neat': 0.59636295, 'attack': 0.22..."
8,A3R01WHD75L6FG,B002B555QQ,0.492050,"{'toy': {'likes': 2.2202773, 'loves': 0.517879..."
9,A3R01WHD75L6FG,B000284ZNI,0.484729,"{'back': {'loves': 0.088577084, 'instantly': 0..."


### Explain one user-item pair

In [2]:
one_user = eval_method.train_set.user_ids[0]
one_item = eval_method.train_set.item_ids[0]
explanation = explainer.explain_one_recommendation_to_user(one_user, one_item)
print(f"For User {one_user} and Item {one_item}:")
print("Explanation:")
print(explanation)

For User A30H2335OM7RD6 and Item B00000IZQE:
Explanation:
{'also': {'bonus': 0.06725946, 'loves': 0.03834136, 'negatives': 0.021852734}, 'toy': {'bonus': 0.21158405, 'loves': 0.11687037, 'negatives': 0.08220584}, 'push': {'bonus': 0.22711197, 'loves': 0.120487414, 'negatives': 0.10063806}}


### Explain one user-item with ref-item

In [4]:
one_user = eval_method.train_set.user_ids[0]
one_item = eval_method.train_set.item_ids[0]
ref_item = eval_method.train_set.item_ids[10]
explanation = explainer.explain_one_with_ref(user_id=one_user, item_id=one_item, ref_item_id=ref_item)
explanation

,user_id,item_id,explanation,ref_item_id,ref_explanation
0,A30H2335OM7RD6,B00000IZQE,"{'also': {'bonus': 0.06725946, 'loves': 0.0383...",1581170769,"{'colors': {'cute': 0.0040567503, 'nice': 0.00..."


In [5]:
# explanation for user-item
print(explanation['explanation'])
# and explanation for user-ref_item
print(explanation['ref_explanation'])

0    {'also': {'bonus': 0.06725946, 'loves': 0.0383...
Name: explanation, dtype: object
0    {'colors': {'cute': 0.0040567503, 'nice': 0.00...
Name: ref_explanation, dtype: object
